<a href="https://colab.research.google.com/github/Tentu16/OPTIFYX/blob/main/Task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

products = pd.DataFrame({
    'product_id': [1, 2, 3, 4, 5],
    'product_name': ['Chair', 'Table', 'Sofa', 'Lamp', 'Bookshelf'],
    'category': ['Furniture', 'Furniture', 'Furniture', 'Lighting', 'Furniture'],
    'description': [
        'A comfortable wooden chair',
        'A large dining table',
        'A modern sofa with cushions',
        'A stylish lamp for reading',
        'A wooden bookshelf with storage'
    ]
})

user_purchases = pd.DataFrame({
    'user_id': [1, 1, 2, 3, 3],
    'product_id': [1, 2, 3, 4, 5]
})

def recommend_content_based(product_id, products, top_n=3):
    vectorizer = CountVectorizer()
    description_matrix = vectorizer.fit_transform(products['description'])

    similarity = cosine_similarity(description_matrix)

    product_index = products[products['product_id'] == product_id].index[0]
    similarity_scores = list(enumerate(similarity[product_index]))

    similar_products = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similar_products = [products.iloc[i[0]]['product_name'] for i in similar_products[1:top_n+1]]
    return similar_products

print("Content-Based Recommendations for 'Chair':")
print(recommend_content_based(1, products))

def recommend_collaborative(user_id, user_purchases, products, top_n=3):

    user_item_matrix = user_purchases.pivot_table(index='user_id', columns='product_id', aggfunc='size', fill_value=0)

    user_similarity = cosine_similarity(user_item_matrix)

    user_index = user_purchases['user_id'].unique().tolist().index(user_id)
    similar_users = list(enumerate(user_similarity[user_index]))
    similar_users = sorted(similar_users, key=lambda x: x[1], reverse=True)[1:]

    similar_user_ids = [user_purchases['user_id'].unique()[i[0]] for i in similar_users[:top_n]]
    similar_user_purchases = user_purchases[user_purchases['user_id'].isin(similar_user_ids)]
    recommended_products = similar_user_purchases['product_id'].unique()

    user_products = user_purchases[user_purchases['user_id'] == user_id]['product_id'].unique()
    final_recommendations = [products[products['product_id'] == pid]['product_name'].values[0]
                             for pid in recommended_products if pid not in user_products]
    return final_recommendations

print("\nCollaborative Filtering Recommendations for User 1:")
print(recommend_collaborative(1, user_purchases, products))

Content-Based Recommendations for 'Chair':
['Bookshelf', 'Table', 'Sofa']

Collaborative Filtering Recommendations for User 1:
['Sofa', 'Lamp', 'Bookshelf']
